In [ ]:
# 1. Load Metrics
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_curve, confusion_matrix

from src.model_training import load_data
from src.model_utils import evaluate_model

prefix = "ecom"  # or "bank"
X_train, y_train, X_test, y_test = load_data(prefix)

model_logreg = joblib.load(f"models/{prefix}_logreg.pkl")
model_xgb = joblib.load(f"models/{prefix}_xgb.pkl")

# 2. Predictions and Metrics
models = {"Logistic Regression": model_logreg, "XGBoost": model_xgb}
results = {}

for name, model in models.items():
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    results[name] = evaluate_model(y_test, y_pred, y_prob)

# 3. Precision-Recall Curve
plt.figure(figsize=(8, 6))
for name, model in models.items():
    y_prob = model.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    plt.plot(recall, precision, label=f"{name}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.grid()
plt.savefig(f"models/{prefix}_pr_curve.png")
plt.show()

# 4. Confusion Matrices
for name, result in results.items():
    cm = result["Confusion Matrix"]
    plt.figure()
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"{name} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.savefig(f"models/{prefix}_{name.lower().replace(' ', '_')}_cm.png")
    plt.show()

# 5. Final Justification
best_model = max(results.items(), key=lambda x: x[1]['AUC-PR'])[0]
print(f"✅ Recommended Model for {prefix.upper()} Dataset: {best_model}")
